In [ ]:
# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
kc_df = pd.read_excel('kc_regional_variation.xlsx', header=[0,1])
kc_df.head()

In [ ]:
kc_df.columns = ['_'.join(col).strip() for col in kc_df.columns.values]
kc_df = kc_df.dropna(how='all').dropna(axis=1, how='all').dropna(axis=0, how='all')
kc_df.head()

In [ ]:
kc_df = kc_df.rename(columns={'Unnamed: 1_level_0_Unnamed: 1_level_1': 'Name'}).drop('Unnamed: 2_level_0_Unnamed: 2_level_1', axis=1)
kc_df = kc_df[kc_df['Name'].notna()]
kc_df.columns

In [ ]:
kc_df.drop(list(kc_df.filter(regex = 'Offences per 100,000 population .1')), axis = 1, inplace=True)
kc_df.rename(columns={'2020/21_Offences per 100,000 population7': '2020/21_Offences per 100,000 population'}, inplace=True)
kc_df.head()

In [ ]:
# clean the Area Name column
area_name = kc_df['Name'].values.copy()
area_name = np.array([name.strip() for name in area_name])

for ind, name in enumerate(area_name):
    discard = name.find(', including GMP')
    if discard != -1:
        area_name[ind] = name[:discard]

area_name

In [ ]:
kc_df['Name'] = area_name
kc_df.loc[kc_df['Name']=='WALES','Name'] = 'Wales'
kc_df.loc[kc_df['Name']=='Surrey5','Name'] = 'Surrey'
kc_df['Name'].values

In [ ]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

url = 'https://opendata.arcgis.com/api/v3/datasets/01fd6b2d7600446d8af768005992f76a_0/downloads/data?format=shp&spatialRefId=27700&where=1%3D1'

with urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('uk_regions_boundaries')

In [ ]:
uk_regions = gpd.read_file('uk_regions_boundaries/NUTS_Level_1_(January_2018)_Boundaries.shp')
uk_regions.head()

In [ ]:
uk_regions = uk_regions[['nuts118nm', 'geometry']]
uk_regions

In [ ]:
area_name = uk_regions['nuts118nm'].values.copy()
for ind, name in enumerate(area_name):
    discard = name.find(' (England)')
    if discard != -1:
        area_name[ind] = name[:discard]
area_name = np.array([name.strip() for name in area_name])
area_name

In [ ]:
area_name[2] = 'Yorkshire and the Humber'
area_name

In [ ]:
uk_regions['Area Name'] = area_name
uk_regions

In [ ]:
kc_df['Name'].values

In [ ]:
uk_regions.loc[uk_regions['Area Name']=='West Midlands', 'Area Name'] = 'West Midlands Region'

In [ ]:
uk_regions_with_kc = pd.merge(uk_regions, kc_df, left_on=uk_regions['Area Name'].str.lower(), 
                              right_on=kc_df['Name'].str.lower(), 
                              how='left')
uk_regions_with_kc

In [ ]:
uk_regions_with_kc.drop([10,11], inplace=True)
uk_regions_with_kc.drop(['key_0', 'Name'], axis=1, inplace=True)

In [ ]:
uk_regions_with_kc.dtypes

In [ ]:
uk_regions_with_kc.columns

In [ ]:
int_col = [uk_regions_with_kc.columns[i*2+1] for i in range(1,len(uk_regions_with_kc.columns)//2)]
float_col = [uk_regions_with_kc.columns[i*2+2] for i in range(1,len(uk_regions_with_kc.columns)//2)]
uk_regions_with_kc[int_col] = uk_regions_with_kc[int_col].astype('int')
uk_regions_with_kc[float_col] = uk_regions_with_kc[float_col].astype('float')
uk_regions_with_kc.dtypes

In [ ]:
flatten_uk_regions_kc_int = pd.melt(uk_regions_with_kc[['nuts118nm', 'geometry', 'Area Name']+int_col], 
                                    id_vars=['nuts118nm', 'geometry', 'Area Name'],
                                    var_name='Year', value_vars=int_col, value_name='Number')
flatten_uk_regions_kc_float = pd.melt(uk_regions_with_kc[['nuts118nm', 'geometry', 'Area Name']+float_col], 
                                      id_vars=['nuts118nm', 'geometry', 'Area Name'],
                                      var_name='Year', value_vars=float_col, value_name='Perc_per_pop')

In [ ]:
flatten_uk_regions_kc_int['Year'] = flatten_uk_regions_kc_int['Year'].apply(int_col.index)
flatten_uk_regions_kc_float['Year'] = flatten_uk_regions_kc_float['Year'].apply(float_col.index)

In [ ]:
flatten_uk_regions_kc = pd.merge(flatten_uk_regions_kc_int, flatten_uk_regions_kc_float, how='inner')
flatten_uk_regions_kc['Year'] = flatten_uk_regions_kc['Year'] + 2008
flatten_uk_regions_kc

In [ ]:
flatten_uk_regions_kc.loc[flatten_uk_regions_kc['Year']==2021, 'Year'] = '2010-21 change'
flatten_uk_regions_kc.dtypes

In [ ]:
flatten_uk_regions_kc.to_file('uk_regions_with_kc/uk_regions_with_kc.shp')

In [ ]:
url = 'https://opendata.arcgis.com/api/v3/datasets/1c8b832d08eb4f77a453156a29f4b86b_0/downloads/data?format=shp&spatialRefId=27700&where=1%3D1'

with urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('police_areas_boundaries')

In [ ]:
police_areas = gpd.read_file('police_areas_boundaries/Police_Force_Areas_Dec_2016_FCB_in_England_and_Wales.shp')
police_areas.head()

In [ ]:
police_areas = police_areas[['pfa16nm', 'geometry']]
kc_df.loc[kc_df['Name']=='Greater Manchester4', 'Name'] = 'Greater Manchester'
police_areas_with_kc = pd.merge(police_areas, kc_df, left_on='pfa16nm', right_on='Name', how='left')
police_areas_with_kc

In [ ]:
police_areas_with_kc.dtypes

In [ ]:
police_areas_with_kc[int_col+float_col] = police_areas_with_kc[int_col+float_col].replace('-', np.nan)
police_areas_with_kc[int_col+float_col] = police_areas_with_kc[int_col+float_col].astype('float')
police_areas_with_kc.dtypes

In [ ]:
flatten_police_areas_with_kc_int = pd.melt(police_areas_with_kc[['pfa16nm', 'geometry', 'Name']+int_col], 
                                           id_vars=['pfa16nm', 'geometry', 'Name'],
                                           var_name='Year', value_vars=int_col, value_name='Number')
flatten_police_areas_with_kc_float = pd.melt(police_areas_with_kc[['pfa16nm', 'geometry', 'Name']+float_col], 
                                             id_vars=['pfa16nm', 'geometry', 'Name'],
                                             var_name='Year', value_vars=float_col, value_name='Perc_per_pop')

flatten_police_areas_with_kc_int['Year'] = flatten_police_areas_with_kc_int['Year'].apply(int_col.index)
flatten_police_areas_with_kc_float['Year'] = flatten_police_areas_with_kc_float['Year'].apply(float_col.index)

flatten_police_areas_with_kc = pd.merge(flatten_police_areas_with_kc_int, flatten_police_areas_with_kc_float, how='inner')
flatten_police_areas_with_kc['Year'] = flatten_police_areas_with_kc['Year'] + 2008
flatten_police_areas_with_kc

In [ ]:
flatten_police_areas_with_kc.loc[flatten_police_areas_with_kc['Year']==2021, 'Year'] = '2010-21 change'

In [ ]:
flatten_police_areas_with_kc.to_file('police_areas_with_kc/police_areas_with_kc.shp')